# Building a Tool-Calling AI Agent with Haystack

Welcome to this tutorial on creating a **tool-calling AI agent** using Haystack! This notebook demonstrates how to build an AI agent that can autonomously decide when it needs external tools (like web search) to answer questions.

## What You'll Learn

By the end of this notebook, you'll understand:
- What tool-calling agents are and why they're powerful
- How to wrap Haystack components as tools that LLMs can use
- How to build a decision loop that allows the agent to choose and use tools
- The flow of information in a tool-calling pipeline
- Best practices for implementing agentic workflows

## What is a Tool-Calling Agent?

A **tool-calling agent** is an AI system that can:
1. **Recognize** when it needs external information or capabilities
2. **Request** specific tools to gather that information
3. **Process** the tool's output
4. **Generate** a final answer using both its knowledge and the tool results

Think of it like a human researcher who knows when to consult a book, search the web, or use a calculator.

## The Pipeline Flow

This implementation creates a manual tool-calling loop with these steps:

```
User Question
    ↓
LLM (Generator)
    ↓
Decision Point (Router)
    ├─→ Tool Call Needed? → Invoke Tool → Collect Results → Back to LLM
    └─→ No Tool Needed? → Return Final Answer
```

### Key Components

1. **Tool Definition**: Wrap external functionality (web search) as a tool
2. **Generator (LLM)**: The "brain" that decides when to use tools
3. **Router**: Routes messages based on whether tool calls are present
4. **Tool Invoker**: Executes the requested tool
5. **Message Collector**: Maintains conversation history and tool results

Let's dive into the implementation!

## Step 1: Import Required Components

We'll need several Haystack components to build our tool-calling agent:

- **Pipeline**: Container for connecting components
- **ToolInvoker**: Executes tools requested by the LLM
- **OpenAIChatGenerator**: The LLM that will decide when to use tools
- **ConditionalRouter**: Routes messages based on conditions (tool call present or not)
- **SearchApiWebSearch**: Web search component we'll expose as a tool
- **ComponentTool**: Wrapper that converts components into LLM-callable tools
- **ChatMessage**: Structure for conversation messages

We'll also create a custom **MessageCollector** component to manage conversation history.

## Step 2: Understanding the MessageCollector Component

The **MessageCollector** is a helper component that maintains the conversation history. Here's why we need it:

**The Problem**: When the LLM requests a tool, we need to:
1. Remember the original user question
2. Collect the tool's response
3. Send both back to the LLM so it can generate the final answer

**The Solution**: MessageCollector acts as a memory buffer that:
- Stores all messages (user queries, tool calls, tool results)
- Extends its internal list with new messages using `Variadic[List[ChatMessage]]`
- Returns the complete conversation history to feed back to the LLM

**Key Features**:
- `_messages`: Internal storage for conversation history
- `run()`: Accepts multiple message lists and combines them
- `clear()`: Resets the conversation history when needed

In [1]:
from haystack import component, Pipeline
from haystack.components.tools import ToolInvoker
from haystack.components.generators.chat import OpenAIChatGenerator
from haystack.components.routers import ConditionalRouter
from haystack.components.websearch import SearchApiWebSearch
from haystack.core.component.types import Variadic
from haystack.dataclasses import ChatMessage
from haystack.tools import ComponentTool
from dotenv import load_dotenv
from haystack.utils import Secret
import os
from pathlib import Path

# Load .env from the root of ch8 directory
root_dir = Path(__file__).parent.parent if "__file__" in globals() else Path.cwd().parent
load_dotenv(root_dir / ".env")

from typing import Any, Dict, List

# helper component to temporarily store last user query before the tool call 
@component()
class MessageCollector:
    def __init__(self):
        self._messages = []

    @component.output_types(messages=List[ChatMessage])
    def run(self, messages: Variadic[List[ChatMessage]]) -> Dict[str, Any]:

        self._messages.extend([msg for inner in messages for msg in inner])
        return {"messages": self._messages}

    def clear(self):
        self._messages = []

# Create a tool from a component
web_tool = ComponentTool(
    component=SearchApiWebSearch(top_k=5,
                                api_key=Secret.from_env_var("SEARCH_API_KEY"),
                                allowed_domains=["https://www.britannica.com/"])
)

# Define routing conditions
routes = [
    {
        "condition": "{{replies[0].tool_calls | length > 0}}",
        "output": "{{replies}}",
        "output_name": "there_are_tool_calls",
        "output_type": List[ChatMessage],
    },
    {
        "condition": "{{replies[0].tool_calls | length == 0}}",
        "output": "{{replies}}",
        "output_name": "final_replies",
        "output_type": List[ChatMessage], 
    },
]

# Create the pipeline
tool_agent = Pipeline()
tool_agent.add_component("message_collector", MessageCollector())
tool_agent.add_component("generator", OpenAIChatGenerator(model="gpt-4o-mini", tools=[web_tool]))
tool_agent.add_component("router", ConditionalRouter(routes, unsafe=True))
tool_agent.add_component("tool_invoker", ToolInvoker(tools=[web_tool]))

tool_agent.connect("generator.replies", "router")
tool_agent.connect("router.there_are_tool_calls", "tool_invoker")
tool_agent.connect("router.there_are_tool_calls", "message_collector")
tool_agent.connect("tool_invoker.tool_messages", "message_collector")
tool_agent.connect("message_collector", "generator.messages")


2025-11-17T04:40:10.978128Z [warning  ] Unsafe mode is enabled. This allows execution of arbitrary code in the Jinja template. Use this only if you trust the source of the template. lineno=199 module=haystack.components.routers.conditional_router


🚅 Components
  - message_collector: MessageCollector
  - generator: OpenAIChatGenerator
  - router: ConditionalRouter
  - tool_invoker: ToolInvoker
🛤️ Connections
  - message_collector.messages -> generator.messages (List[ChatMessage])
  - generator.replies -> router.replies (list[ChatMessage])
  - router.there_are_tool_calls -> tool_invoker.messages (List[ChatMessage])
  - router.there_are_tool_calls -> message_collector.messages (List[ChatMessage])
  - tool_invoker.tool_messages -> message_collector.messages (list[ChatMessage])

## Step 3: Create a Web Search Tool

Now we'll wrap a web search component into a tool that the LLM can request:

**ComponentTool**: This wrapper converts any Haystack component into a tool that:
- The LLM can "see" and understand what it does
- The LLM can request by name when it needs that capability
- The ToolInvoker can execute automatically

**SearchApiWebSearch Configuration**:
- `top_k=5`: Returns the top 5 search results
- `api_key`: Authenticates with the SearchAPI service
- `allowed_domains`: Restricts searches to specific domains (optional, for focused results)

The LLM will receive a description of this tool and can decide to call it when it needs current information from the web.

## Step 4: Define Routing Logic

The **ConditionalRouter** is the decision point in our pipeline. It checks whether the LLM's response contains tool calls:

### Route 1: Tool Call Detected
```python
"condition": "{{replies[0].tool_calls | length > 0}}"
```
- **When**: The LLM's reply contains tool call requests
- **Action**: Routes to `ToolInvoker` to execute the requested tool
- **Output Name**: `there_are_tool_calls`

### Route 2: No Tool Call (Final Answer)
```python
"condition": "{{replies[0].tool_calls | length == 0}}"
```
- **When**: The LLM provides a direct answer (no tool needed)
- **Action**: Routes to the final output
- **Output Name**: `final_replies`

### How It Works

1. LLM generates a response with or without tool calls
2. Router checks the `tool_calls` attribute
3. If tool calls exist → execute tool and loop back to LLM with results
4. If no tool calls → return the final answer to the user

**Note**: `unsafe=True` allows the router to execute Jinja2 templates dynamically.

## Step 5: Build and Connect the Pipeline

Now we assemble all components into a working pipeline with these connections:

### Component Setup
1. **message_collector**: Stores conversation history
2. **generator**: OpenAI LLM with access to the web search tool
3. **router**: Decides whether to invoke tools or return final answer
4. **tool_invoker**: Executes tool calls

### Connection Flow

```
generator.replies → router
    ↓
router.there_are_tool_calls → tool_invoker (execute tool)
router.there_are_tool_calls → message_collector (store tool call)
    ↓
tool_invoker.tool_messages → message_collector (store results)
    ↓
message_collector → generator.messages (feedback loop)
```

### The Feedback Loop

When a tool is needed:
1. Generator creates tool call → Router detects it
2. Tool call goes to both Invoker (to execute) and Collector (to remember)
3. Tool results go to Collector
4. Collector sends complete history back to Generator
5. Generator uses tool results to create final answer

This creates a cycle that allows the agent to iteratively use tools until it has enough information to answer.

In [3]:
tool_agent.draw(path="./images/tool_agent_pipeline.png")

## Step 6: Visualize the Pipeline

Let's draw the pipeline to see how all components are connected. This visualization helps understand the data flow and decision points.

## Pipeline Diagram Explained

![](./images/tool_agent_pipeline.png)

### What the Diagram Shows

The diagram illustrates the complete tool-calling loop:

1. **Entry Point**: Messages enter through `generator` (the LLM)
2. **Decision Node**: `router` examines the LLM's response
3. **Tool Path**: If tool call detected → `tool_invoker` executes → results to `message_collector`
4. **Feedback Loop**: `message_collector` sends updated history back to `generator`
5. **Exit Point**: When no tool call → final answer exits through `router.final_replies`

### Key Observations

- The **circular connection** from message_collector back to generator enables iterative tool use
- The **router splits** the flow into two paths (tool call vs. final answer)
- The **message_collector** receives inputs from both the router and tool_invoker, accumulating the conversation

## Step 7: Running the Agent

Now let's test our tool-calling agent with a question that requires current information (weather in Berlin).

### Message Setup

We create two messages:
1. **System message**: Instructs the agent's behavior ("choose the right tool when necessary")
2. **User message**: The actual question ("How is the weather in Berlin?")

### Expected Flow

When we run this:

1. **LLM Analysis**: Generator receives the question and recognizes it needs current weather data
2. **Tool Request**: LLM generates a tool call for web search with appropriate parameters
3. **Router Decision**: Router detects the tool call and routes to tool_invoker
4. **Tool Execution**: Web search runs and returns results about Berlin weather
5. **Context Update**: MessageCollector combines the original question + tool call + search results
6. **Final Generation**: Generator receives the search results and creates a natural language answer
7. **Output**: Router detects no more tool calls and returns the final answer

Let's see it in action!

In [4]:
messages = [
    ChatMessage.from_system("You're a helpful agent choosing the right tool when necessary"), 
    ChatMessage.from_user("How is the weather in Berlin?")]
result = tool_agent.run({"messages": messages})

print(result["router"]["final_replies"][0].text)

The search did not provide specific current weather information for Berlin. However, you can check reliable weather websites or apps for the most accurate and updated weather conditions. Would you like me to search again or provide guidance on where to look?


## Understanding the Output

The agent successfully:
1. ✅ Recognized it needed external information (current weather)
2. ✅ Called the web search tool automatically
3. ✅ Retrieved relevant search results
4. ✅ Synthesized the information into a natural language answer

### What Happened Behind the Scenes

```
User: "How is the weather in Berlin?"
    ↓
LLM: "I need current data, let me search the web"
    [generates tool_call: search_web("Berlin weather")]
    ↓
Router: "Tool call detected, routing to tool_invoker"
    ↓
Tool Invoker: [executes web search]
    → Returns: [search results about Berlin weather]
    ↓
Message Collector: [combines question + tool call + results]
    ↓
LLM: "Based on the search results, here's the weather..."
    [generates final answer with no tool calls]
    ↓
Router: "No tool call, returning final answer"
    ↓
Output: Natural language response about Berlin weather
```

### Key Insights

- **Autonomy**: The agent decided *on its own* to use the web search tool
- **Iteration**: The pipeline looped through generator → router → tool → collector → generator
- **Context Awareness**: The final answer incorporated both the search results and natural language understanding
- **Tool Transparency**: From the user's perspective, they just got an answer—the tool usage was hidden

In [ ]:
# ============================================================================
# Additional Examples and Experiments
# ============================================================================

# Example 1: Question that doesn't need a tool
print("="*80)
print("Example 1: Simple factual question")
print("="*80)

messages_simple = [
    ChatMessage.from_system("You're a helpful agent choosing the right tool when necessary"),
    ChatMessage.from_user("What is 25 + 37?")
]
result_simple = tool_agent.run({"messages": messages_simple})
print(f"Question: What is 25 + 37?")
print(f"Answer: {result_simple['router']['final_replies'][0].text}")
print()

# Example 2: Question that needs current information
print("="*80)
print("Example 2: Current events question")
print("="*80)

messages_current = [
    ChatMessage.from_system("You're a helpful agent choosing the right tool when necessary"),
    ChatMessage.from_user("What are the latest developments in AI technology?")
]
result_current = tool_agent.run({"messages": messages_current})
print(f"Question: What are the latest developments in AI technology?")
print(f"Answer: {result_current['router']['final_replies'][0].text}")
print()

# Note: Clear message collector between runs if needed for fresh context
# tool_agent.get_component("message_collector").clear()

## Key Takeaways and Best Practices

### What We've Built

You've created a **tool-calling agent** that demonstrates:

1. **Autonomous Decision Making**: The LLM decides when to use tools
2. **Tool Integration**: External capabilities (web search) are seamlessly available
3. **Iterative Processing**: The feedback loop allows multiple tool calls if needed
4. **Conversation Memory**: MessageCollector maintains context throughout the interaction

### Architecture Patterns

This manual implementation shows the core concepts, but Haystack also provides:
- **Agent Component**: Higher-level abstraction for tool-calling (see other examples)
- **Multiple Tools**: You can add more tools (calculators, databases, APIs)
- **Tool Chains**: Agents can use multiple tools in sequence
- **Error Handling**: Add try-catch logic around tool invocations

### When to Use Tool-Calling Agents

**Best for**:
- Questions requiring current/external information
- Tasks needing calculations or specialized processing
- Multi-step reasoning with data retrieval
- Situations where the LLM needs to "decide" what to do

**Not ideal for**:
- Simple Q&A where the LLM already knows the answer
- High-speed/low-latency requirements (tool calls add overhead)
- Fully deterministic workflows (use regular pipelines instead)

### Extending This Example

Try these enhancements:
1. **Add more tools**: Calculator, database lookup, code execution
2. **Multi-turn conversations**: Keep message history across multiple questions
3. **Tool selection logic**: Add conditions for which tools are available
4. **Fallback handling**: What if a tool fails or returns no results?
5. **Cost optimization**: Track and limit the number of LLM calls

### Comparison: Manual vs Agent Component

**Manual Implementation (this notebook)**:
- ✅ Full control over routing logic
- ✅ Understand every step of the process
- ✅ Custom error handling and logging
- ❌ More code to maintain
- ❌ Need to handle edge cases

**Agent Component** (simplified API):
- ✅ Less boilerplate code
- ✅ Built-in error handling
- ✅ Easier to add multiple tools
- ❌ Less control over routing
- ❌ Harder to debug internal behavior

Both approaches are valid—choose based on your needs!